<a href="https://colab.research.google.com/github/FilipeMSouza/ml-2023-1-trabalho-final/blob/master/music_genre_classificationl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings

warnings.filterwarnings("ignore")

In [43]:
# Carregar os dados
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [44]:
# Visualizar as primeiras linhas dos dados de treinamento
display(df_train.head())

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [45]:
# Verificar informações sobre o DataFrame de treinamento
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Artist Name         17996 non-null  object 
 1   Track Name          17996 non-null  object 
 2   Popularity          17568 non-null  float64
 3   danceability        17996 non-null  float64
 4   energy              17996 non-null  float64
 5   key                 15982 non-null  float64
 6   loudness            17996 non-null  float64
 7   mode                17996 non-null  int64  
 8   speechiness         17996 non-null  float64
 9   acousticness        17996 non-null  float64
 10  instrumentalness    13619 non-null  float64
 11  liveness            17996 non-null  float64
 12  valence             17996 non-null  float64
 13  tempo               17996 non-null  float64
 14  duration_in min/ms  17996 non-null  float64
 15  time_signature      17996 non-null  int64  
 16  Clas

In [46]:
# Resumo estatístico do DataFrame de treinamento
df_train.describe()

,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
count,17568.000000,17996.000000,17996.000000,15982.000000,17996.000000,17996.000000,17996.000000,17996.000000,13619.000000,17996.000000,17996.000000,17996.000000,1.799600e+04,17996.000000,17996.000000
mean,44.512124,0.543433,0.662777,5.952447,-7.910660,0.636753,0.079707,0.247082,0.177562,0.196170,0.486208,122.623294,2.007445e+05,3.924039,6.695821
std,17.426928,0.166268,0.235373,3.196854,4.049151,0.480949,0.083576,0.310632,0.304048,0.159212,0.240195,29.571527,1.119891e+05,0.361618,3.206073
min,1.000000,0.059600,0.000020,1.000000,-39.952000,0.000000,0.022500,0.000000,0.000001,0.011900,0.018300,30.557000,5.016500e-01,1.000000,0.000000
25%,33.000000,0.432000,0.509000,3.000000,-9.538000,0.000000,0.034800,0.004300,0.000089,0.097500,0.297000,99.620750,1.663370e+05,4.000000,5.000000
50%,44.000000,0.545000,0.700000,6.000000,-7.016000,1.000000,0.047400,0.081400,0.003910,0.129000,0.481000,120.065500,2.091600e+05,4.000000,8.000000
75%,56.000000,0.659000,0.860000,9.000000,-5.189000,1.000000,0.083000,0.434000,0.200000,0.258000,0.672000,141.969250,2.524900e+05,4.000000,10.000000
max,100.000000,0.989000,1.000000,11.000000,1.355000,1.000000,0.955000,0.996000,0.996000,1.000000,0.986000,217.416000,1.477187e+06,5.000000,10.000000


In [47]:
# Verificar o número de valores únicos em cada coluna
for col in df_train.columns:
    print(col + ": " + str(df_train[col].nunique()))

Artist Name: 9149
Track Name: 15129
Popularity: 100
danceability: 909
energy: 1208
key: 11
loudness: 9002
mode: 2
speechiness: 1194
acousticness: 3930
instrumentalness: 4253
liveness: 1459
valence: 1320
tempo: 13562
duration_in min/ms: 14121
time_signature: 4
Class: 11


In [48]:
# Verificar a contagem de valores ausentes em cada coluna
df_train.isnull().sum()

Artist Name              0
Track Name               0
Popularity             428
danceability             0
energy                   0
key                   2014
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      4377
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                    0
dtype: int64

In [49]:
for col in df_train.columns[df_train.isna().any()].tolist():
    print(col + ": " + str(df_train[col].nunique()))

Popularity: 100
key: 11
instrumentalness: 4253


In [50]:
df_train["key"].mode()

0    7.0
Name: key, dtype: float64

In [51]:
# Preencher os valores ausentes com as modas, medianas ou estratégias adequadas
mode = df_train["Popularity"].mode()
df_train["Popularity"] = df_train["Popularity"].fillna(mode[0])
mode = df_train["key"].mode()
df_train["key"] = df_train["key"].fillna(mode[0])
median = df_train["instrumentalness"].median()
df_train["instrumentalness"] =  df_train["instrumentalness"].fillna(median)

In [52]:
# Verificar novamente a contagem de valores ausentes
df_train.isnull().sum()

Artist Name           0
Track Name            0
Popularity            0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_in min/ms    0
time_signature        0
Class                 0
dtype: int64

In [53]:
# Discretizar a coluna "Popularity" em 5 bins
df_train["Popularity"] = pd.cut(df_train["Popularity"], bins=5, labels=False)
df_test["Popularity"] = pd.cut(df_test["Popularity"], 5, labels=False)

In [54]:
# Converter colunas categóricas em tipo "category"
df_train[["key", "mode", "time_signature", "Popularity"]] = df_train[["key", "mode", "time_signature", "Popularity"]].astype("category")
df_test[["key", "mode", "time_signature", "Popularity"]] = df_test[["key", "mode", "time_signature", "Popularity"]].astype("category")

In [55]:
# Separar as features e o target
Y = df_train["Class"]
X = df_train.drop(["Class", "Artist Name", "Track Name"], axis=1)
cat_cols = X.select_dtypes("category").columns
num_cols = X.select_dtypes(include=np.number).columns
print(cat_cols)
print(num_cols)

Index(['Popularity', 'key', 'mode', 'time_signature'], dtype='object')
Index(['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms'],
      dtype='object')


In [56]:
# Dividir os dados em treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)

In [57]:
# Padronizar as features numéricas
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train[num_cols])
x_test_scaled = scaler.transform(x_test[num_cols])

In [58]:
# Criar DataFrames com as features padronizadas e as colunas categóricas
scaled_features_df_train = pd.DataFrame(x_train_scaled, index=x_train[num_cols].index, columns=x_train[num_cols].columns)
scaled_features_df_test = pd.DataFrame(x_test_scaled, index=x_test[num_cols].index, columns=x_test[num_cols].columns)
df_train_prepro = pd.concat((scaled_features_df_train, x_train[cat_cols]), axis=1)
df_test_prepro = pd.concat((scaled_features_df_test, x_test[cat_cols]), axis=1)



In [59]:
#Definir os modelos a serem avaliados
models = [
    ("log_reg", LogisticRegression(multi_class="multinomial", max_iter=1000)),
    ("naive_bay", GaussianNB()),
    ("kn_clf", KNeighborsClassifier()),
    ("dtree", DecisionTreeClassifier()),
    ("rforest", RandomForestClassifier()),
]

In [60]:

# Avaliar os modelos usando métricas de desempenho
df_metrics = pd.DataFrame(index=["name", "accuracy", "precision", "recall", "f1_score"])
n = 0
for name, model in models:
    model.fit(df_train_prepro, y_train)
    y_pred = model.predict(df_test_prepro)
    metrics = [name]
    metrics.append(accuracy_score(y_test, y_pred))
    metrics.append(precision_score(y_test, y_pred, average="weighted"))
    metrics.append(recall_score(y_test, y_pred, average="weighted"))
    metrics.append(f1_score(y_test, y_pred, average="weighted"))
    df_metrics[n] = metrics
    n += 1

In [61]:
# Exibir as métricas de desempenho dos modelos
display(df_metrics)

,0,1,2,3,4
name,log_reg,naive_bay,kn_clf,dtree,rforest
accuracy,0.497569,0.453674,0.417002,0.366857,0.510765
precision,0.464935,0.437683,0.418729,0.381028,0.498202
recall,0.497569,0.453674,0.417002,0.366857,0.510765
f1_score,0.469316,0.43094,0.416749,0.373104,0.496696
